In [1]:
from __future__ import print_function
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Define the neural network
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['data']
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    return out_layer

In [3]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    logits = neural_net(features)
    
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [4]:
# Evaluate the Model
def evaluate_model(x, y):
    # Define the input function for evaluating
    input_fn = tf.estimator.inputs.numpy_input_fn(x={'data': x}, y=y, batch_size=batch_size, shuffle=False)
    
    # Use the Estimator 'evaluate' method
    model.evaluate(input_fn)

In [5]:
# Predict text data
def predict_data(text_data):
    data = vectorizer2.transform(text_data)
    data = data.todense()
    
    # Prepare the input data
    input_fn = tf.estimator.inputs.numpy_input_fn(x={'data': data}, shuffle=False)

    # Use the model to predict the images class
    preds = list(model.predict(input_fn))
    
    for i in range(len(preds)):
        print('Text:\n', text_data[i])
        print('Prediction Label:\n', ng_train.target_names[preds[i]])
        print('-------------------------------------------------------')

In [6]:
# Fetch data
ng_train = fetch_20newsgroups(subset='train')
ng_test = fetch_20newsgroups(subset='test')

# Training and testing data
X_train = ng_train.data
X_test = ng_test.data

# Set of label names for training and testing data
# ng_train.target_names
# ng_test.target_names

# Training and testing labels, where each label is associated with a number corresponding to the index in target_names
# ng_train.target
# ng_test.target

# Training and testing labels
y_train = []
y_test = []

for i in range(len(X_train)):
    y_train.append(ng_train.target_names[ng_train.target[i]])
for i in range(len(X_test)):
    y_test.append(ng_test.target_names[ng_test.target[i]])

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

11314
7532
11314
7532


In [7]:
# Converting text to vectors
tfidf = TfidfVectorizer()
tfidf.fit(X_train)
vect_X_train = tfidf.transform(X_train)
vect_X_test = tfidf.transform(X_test)

print(vect_X_train.shape)
print(vect_X_test.shape)

In [8]:
# Reduce data to top K features
K = 1000
model = SelectKBest(chi2, K)

model.fit(vect_X_train, y_train)
importances = model.scores_      # Scores of each feature

indices = np.argsort(importances)[::-1]
features = tfidf.get_feature_names()

# Store top k features
new_features = []
for f in range(K):
    new_features.append(features[indices[f]])

# Convert train and test data to vectors based on k features
new_tfidf = TfidfVectorizer(stop_words='english', vocabulary=new_features)
new_tfidf.fit(X_train)

vect_X_train = new_tfidf.transform(X_train)
vect_X_test = new_tfidf.transform(X_test)

In [9]:
X_train = vect_X_train.todense()
X_test = vect_X_test.todense()
y_train = np.array(y_train)
y_test = np.array(y_test)

In [10]:
# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = K # MNIST data input (img shape: 28*28)
num_classes = 20 # MNIST total classes (0-9 digits)

In [11]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'data': X_train}, y=y_train,
    batch_size=batch_size, num_epochs=None, shuffle=True)

In [12]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Ashton\\AppData\\Local\\Temp\\tmp7eslsg_u', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025E71A124E0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
# Train the Model
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Ashton\AppData\Local\Temp\tmp7eslsg_u\model.ckpt.
INFO:tensorflow:loss = 2.997783130813092, step = 1
INFO:tensorflow:global_step/sec: 106.08
INFO:tensorflow:loss = 2.901840024661614, step = 101 (0.944 sec)
INFO:tensorflow:global_step/sec: 108.781
INFO:tensorflow:loss = 2.7273493966411824, step = 201 (0.919 sec)
INFO:tensorflow:global_step/sec: 103.351
INFO:tensorflow:loss = 2.4547067033405647, step = 301 (0.968 sec)
INFO:tensorflow:global_step/sec: 96.4673
INFO:tensorflow:loss = 2.0258074344170627, step = 401 (1.038 sec)
INFO:tensorflow:global_step/sec: 86.8007
INFO:tensorflow:loss = 1.634514477711623, step = 501 (1.154 sec)
INFO:tensorflow:global_step/sec: 105.219
INFO:tensorflow:loss = 1.174969220

In [14]:
evaluate_model(X_train, y_train)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-23-00:12:45
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Ashton\AppData\Local\Temp\tmp7eslsg_u\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-23-00:12:45
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.81951565, global_step = 1000, loss = 0.71911603


In [15]:
predict_data(['Discovering new species of animals is not easy', 'Space exploration is my favorite past time', 'I have never hit a home run'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Ashton\AppData\Local\Temp\tmp7eslsg_u\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Text:
 Discovering new species of animals is not easy
Prediction Label:
 sci.med
-------------------------------------------------------
Text:
 Space exploration is my favorite past time
Prediction Label:
 sci.space
-------------------------------------------------------
Text:
 I have never hit a home run
Prediction Label:
 rec.sport.baseball
-------------------------------------------------------


In [16]:
evaluate_model(X_test, y_test)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-23-00:12:46
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Ashton\AppData\Local\Temp\tmp7eslsg_u\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-23-00:12:46
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.7368561, global_step = 1000, loss = 0.9465906


In [17]:
predict_data(['I need some guidance in my life, I am going to Church tomorrow', 'I want to sell my soul', 'This screen is sharp and bright'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Ashton\AppData\Local\Temp\tmp7eslsg_u\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Text:
 I need some guidance in my life, I am going to Church tomorrow
Prediction Label:
 soc.religion.christian
-------------------------------------------------------
Text:
 I want to sell my soul
Prediction Label:
 misc.forsale
-------------------------------------------------------
Text:
 This screen is sharp and bright
Prediction Label:
 comp.sys.mac.hardware
-------------------------------------------------------
